# EDA – Consommation de Pellets & Météo
## 0. Question métier
**Comment les conditions météorologiques influencent-elles la durée de chauffe, la quantité de pellet consommée et le coût final pour l'utilisateur ?**

In [10]:
import sys

!{sys.executable} -m pip install seaborn

In [19]:
# 1. Chargement des données
import pandas as pd
import seaborn as sns
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt

DATA_METEO_PATH = os.getenv("METEO_CSV_PATH", "data/data_meteo.csv")
DATA_CONS_PATH = os.getenv("CONS_CSV_PATH", "consommation.csv")
LEGACY_METEO = "meteo.csv"
FALLBACK_CONS = "data/valdoise_pellet_dataset.csv"


def _build_root_hints() -> list[Path]:
    """Return plausible workspace roots so relative CSV paths resolve even from notebooks."""
    hints: list[Path] = []
    env_root = os.getenv("THERMOSTATS_ROOT")
    if env_root:
        try:
            env_path = Path(env_root).expanduser().resolve()
            hints.append(env_path)
        except OSError:
            pass
    cwd = Path.cwd().resolve()
    for candidate in (cwd, cwd.parent, cwd.parent.parent):
        if candidate and candidate not in hints:
            hints.append(candidate)
    return hints


ROOT_HINTS = _build_root_hints()


def _candidate_paths(raw_path: str) -> list[Path]:
    path_obj = Path(raw_path).expanduser()
    if path_obj.is_absolute():
        return [path_obj]
    candidates: list[Path] = []
    seen: set[str] = set()
    for base in ROOT_HINTS:
        candidate = (base / path_obj).resolve()
        key = str(candidate)
        if key in seen:
            continue
        seen.add(key)
        candidates.append(candidate)
    return candidates or [path_obj]


def load_csv(path: str, *, fallback_paths: list[str] | None = None) -> pd.DataFrame:
    """Charge un CSV en testant plusieurs emplacements sans échouer si on lance depuis notebooks."""
    search_paths = [path] + list(fallback_paths or [])
    for raw in search_paths:
        if not raw:
            continue
        for candidate in _candidate_paths(raw):
            if not candidate.exists():
                continue
            try:
                df = pd.read_csv(candidate)
            except Exception as exc:  # pragma: no cover - informationnel uniquement
                print(f"Impossible de lire {candidate}: {exc}")
                continue
            if df.empty:
                print(f"{candidate} est vide, tentative sur un autre fichier.")
                continue
            if os.path.normpath(str(candidate)) != os.path.normpath(raw):
                print(f"Chargement des données depuis {candidate}")
            return df
    return pd.DataFrame()


df_meteo = load_csv(DATA_METEO_PATH, fallback_paths=[LEGACY_METEO])
if df_meteo.empty:
    print("Aucune donnée météo disponible (data/data_meteo.csv ou meteo.csv).")


df_cons = load_csv(DATA_CONS_PATH, fallback_paths=[FALLBACK_CONS])
if df_cons.empty:
    print(f"Aucune donnée de consommation disponible dans {DATA_CONS_PATH} ni dans les fallbacks.")

# Afficher un aperçu (ok si DataFrame vide)
df_meteo.head(), df_cons.head()

Chargement des données depuis C:\Users\MAF\Documents\thermostats\data\data_meteo.csv
Chargement des données depuis C:\Users\MAF\Documents\thermostats\data\valdoise_pellet_dataset.csv


(                  time  temp_ext  wind  humidity  solar_radiation  temp_eff
 0  2025-11-16 00:00:00      11.8   6.0      87.0              0.0     10.60
 1  2025-11-16 01:00:00      12.0   6.9      92.0              0.0     10.62
 2  2025-11-16 02:00:00      11.6   4.3      91.0              0.0     10.74
 3  2025-11-16 03:00:00      11.4   7.1      91.0              0.0      9.98
 4  2025-11-16 04:00:00      10.5   6.4      91.0              0.0      9.22,
          date  temp_avg_c  temp_min_c  temp_max_c  wind_avg_kmh  \
 0  2025-08-18       21.76        15.2        27.9         10.07   
 1  2025-08-19       22.30        16.2        29.5          6.98   
 2  2025-08-20       19.36        16.6        23.1          9.57   
 3  2025-08-21       18.43        14.1        23.6         13.28   
 4  2025-08-22       16.58        11.5        20.6          9.15   
 
    duration_hours  pellet_bags  pellet_cost_eur  
 0            1.86        0.133             0.64  
 1            1.03       

## 2. Nettoyage rapide

In [12]:
df_meteo.info()
df_cons.info()

# Nettoyage minimal
df_meteo = df_meteo.drop_duplicates()
df_cons = df_cons.drop_duplicates()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [13]:
# Harmonisation minimale des colonnes clés
from collections import defaultdict

def ensure_columns(df: pd.DataFrame, aliases: dict[str, list[str]]) -> pd.DataFrame:
    df = df.copy()
    if df.empty:
        return df
    for canonical, variations in aliases.items():
        if canonical in df.columns:
            continue
        for alt in variations:
            if alt in df.columns:
                df[canonical] = df[alt]
                break
    return df

meteo_aliases = {
    "temperature": ["temp", "temp_ext", "temp_avg_c", "temperature_c", "t_air"],
}
cons_aliases = {
    "temperature_ext": ["temp", "temp_exterieure", "temperature", "t_air", "temp_avg_c"],
    "duree_chauffe": ["duree", "durée", "hours_on", "duration_hours"],
    "consommation": ["consommation_kg", "bags", "pellets", "pellet_bags"],
    "cout": ["cost", "cout_cycle", "cout_total", "pellet_cost_eur"],
}

df_meteo = ensure_columns(df_meteo, meteo_aliases)
df_cons = ensure_columns(df_cons, cons_aliases)

missing_report = defaultdict(list)
for label, df, expected in [
    ("df_meteo", df_meteo, ["temperature"]),
    ("df_cons", df_cons, ["temperature_ext", "duree_chauffe", "consommation", "cout"]),
]:
    for col in expected:
        if col not in df.columns:
            missing_report[label].append(col)

for name, missing in missing_report.items():
    if missing:
        print(f"⚠️ Colonnes manquantes dans {name} : {missing}")
    else:
        print(f"✅ Colonnes requises présentes pour {name}.")

⚠️ Colonnes manquantes dans df_meteo : ['temperature']
⚠️ Colonnes manquantes dans df_cons : ['temperature_ext', 'duree_chauffe', 'consommation', 'cout']


## 3. Analyse univariée

In [14]:
if df_meteo.empty:
    print("Aucune donnée météo disponible pour tracer la distribution des températures.")
elif "temperature" not in df_meteo.columns:
    print("La colonne 'temperature' est absente de df_meteo après harmonisation.")
else:
    plt.figure(figsize=(8, 4))
    sns.histplot(df_meteo["temperature"].dropna(), kde=True)
    plt.title("Distribution des températures (°C)")
    plt.xlabel("Température (°C)")
    plt.tight_layout()
    plt.show()

Aucune donnée météo disponible pour tracer la distribution des températures.


## 4. Analyse bivariée

In [15]:
required_cols = {"temperature_ext", "duree_chauffe"}
if df_cons.empty:
    print("Aucune donnée de consommation disponible pour l'analyse bivariée.")
elif not required_cols.issubset(df_cons.columns):
    print(f"Colonnes manquantes pour le scatterplot : {sorted(required_cols - set(df_cons.columns))}")
else:
    plt.figure(figsize=(8, 4))
    sns.scatterplot(data=df_cons, x="temperature_ext", y="duree_chauffe")
    plt.title("Température extérieure vs Durée de chauffe")
    plt.xlabel("Température extérieure (°C)")
    plt.ylabel("Durée de chauffe (h)")
    plt.tight_layout()
    plt.show()

Aucune donnée de consommation disponible pour l'analyse bivariée.


## 5. Corrélations

In [16]:
if df_cons.empty:
    print("Aucune donnée de consommation disponible pour la heatmap des corrélations.")
else:
    numeric_df = df_cons.select_dtypes(include="number")
    if numeric_df.shape[1] < 2:
        print("Pas assez de colonnes numériques pour calculer une matrice de corrélation.")
    else:
        corr = numeric_df.corr()
        plt.figure(figsize=(8, 6))
        sns.heatmap(corr, annot=True, cmap="viridis", fmt=".2f")
        plt.title("Heatmap des corrélations")
        plt.tight_layout()
        plt.show()

Aucune donnée de consommation disponible pour la heatmap des corrélations.


## 6. KPIs calculés

In [17]:
kpi = {}
if df_cons.empty:
    print("Impossible de calculer des KPI : df_cons est vide.")
else:
    mapping = {
        "Consommation_moyenne": "consommation",
        "Duree_chauffe_moyenne": "duree_chauffe",
        "Cout_moyen_cycle": "cout",
    }
    missing = [col for col in mapping.values() if col not in df_cons.columns]
    if missing:
        print(f"Colonnes manquantes pour les KPI : {missing}")
    for label, column in mapping.items():
        if column in df_cons.columns:
            kpi[label] = df_cons[column].mean()
    if not kpi:
        print("Aucun KPI calculable avec les colonnes présentes.")
        kpi = {}

kpi

Impossible de calculer des KPI : df_cons est vide.


{}

## 7 bis. Synthèse par tranche de température
Comparer les grandeurs (temps de chauffe, consommation, coût) par plage de température extérieure permet d'identifier les zones les plus énergivores.

In [18]:
summary_cols = ["temperature_ext", "consommation", "duree_chauffe", "cout"]
missing_for_summary = [col for col in ["temperature_ext"] if col not in df_cons.columns]
if df_cons.empty:
    print("Impossible de produire la synthèse : df_cons est vide.")
elif missing_for_summary:
    print(f"Colonnes manquantes pour construire les plages de température : {missing_for_summary}.")
else:
    working = df_cons.dropna(subset=["temperature_ext"]).copy()
    if working.empty or working["temperature_ext"].nunique() < 2:
        print("Données insuffisantes pour construire des tranches de température.")
    else:
        quantile_count = min(4, working["temperature_ext"].nunique())
        if quantile_count < 2:
            print("Nombre de quantiles insuffisant pour segmenter les températures.")
        else:
            working["temp_bin"] = pd.qcut(
                working["temperature_ext"],
                q=quantile_count,
                duplicates="drop",
            )
            agg_cols = [col for col in summary_cols if col in working.columns]
            summary = (
                working.groupby("temp_bin")[agg_cols]
                .mean()
                .rename(columns={
                    "temperature_ext": "Température moy. (°C)",
                    "consommation": "Consommation moy. (kg)",
                    "duree_chauffe": "Durée moy. (h)",
                    "cout": "Coût moyen (€)",
                })
                .round(2)
                .reset_index()
            )
            display(summary)

Impossible de produire la synthèse : df_cons est vide.


## 7. Conclusion EDA
Les relations observées montrent une influence claire de la météo sur la consommation et le coût de chauffage.